In [0]:
DEBUG = False

DESIGN_PATH = '/raid/pfs/data/targeting/dSph/bootes/netflow/bootes_1/'

# Validate PFS Design Files

In [0]:
import os, sys
import re
from glob import glob
from datetime import datetime, timedelta
import numpy as np
import matplotlib.pyplot as plt

In [0]:
from pfs.datamodel import PfsDesign

In [0]:
def get_pfsDesignId(fn):
    #fn = design_files[0]
    return int(re.search(r'pfsDesign-0x([0-9a-fA-F]{16})\.fits', fn).group(1), 16)

In [0]:
# Find design files
design_ids = []
for design_file in glob(os.path.join(DESIGN_PATH, 'pfsDesign-*.fits')):
    design_ids.append(get_pfsDesignId(design_file))
    print(design_file, hex(design_ids[-1]))

In [0]:
pfsDesignId = design_ids[0]
pfsDesign = PfsDesign.read(pfsDesignId, dirName=DESIGN_PATH)

In [0]:
# Verify that the instrument rotator is within limits
from pfs.utils.coordinates import DistortionCoefficients as DCoeff

ra = pfsDesign.raBoresight
dec = pfsDesign.decBoresight
posang = pfsDesign.posAng
obs_time = datetime(2025, 1, 24, 3, 30, 0) + timedelta(hours=10)

az, el, inr = DCoeff.radec_to_subaru(ra, dec, posang, obs_time,
                                     epoch=2000.0, pmra=0.0, pmdec=0.0, par=1e-6)

if inr < -174 or inr > 174:
    raise ValueError(f'Instrument rotator angle {inr} is out of range')

print(f'ra={ra}, dec={dec}, posang={posang}, obs_time={obs_time}')
print(f'az={az}, el={el}, inr={inr}')